In [1]:
# Core Libraries
import pandas as pd 
import numpy as np

# Prediction Libraries
from xgboost import XGBRegressor
from mlforecast import MLForecast
from window_ops.rolling import rolling_mean, rolling_max, rolling_min
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import DistributionLoss

c:\Users\Batuhan\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] Belirtilen yordam bulunamadı'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Cleaning Data Sets

In [21]:
BrentDF = pd.read_csv("DataSet/brent.csv")
BrentDF["Tarih"] = pd.to_datetime(BrentDF["Tarih"])
BrentDF["Tarih"] = pd.DatetimeIndex(BrentDF["Tarih"].dt.strftime("%Y-%m-%d"))
BrentDF = BrentDF.iloc[:,:-1]
BrentDF.set_index("Tarih", inplace=True)
BrentDF = BrentDF.reset_index()
BrentDF.rename(columns={"Tarih":"Date"},inplace= True)
BrentDF

,Date,Ürün,Avrupa Birliği Para Birimi,AB Piyasa Fiyatı,AB Piyasa Fiyatı- Yüksek,AB Piyasa Fiyatı- Düşük
0,2023-02-01,ICE BRENT,$/BBL,82.84,86.21,82.37
1,2023-01-31,ICE BRENT,$/BBL,84.49,85.25,83.73
2,2023-01-30,ICE BRENT,$/BBL,84.90,87.48,84.66
3,2023-01-29,ICE BRENT,$/BBL,86.66,88.95,85.69
4,2023-01-28,ICE BRENT,$/BBL,86.66,88.95,85.69
...,...,...,...,...,...,...
1852,2018-01-06,ICE BRENT,$/BBL,67.62,68.11,67.26
1853,2018-01-05,ICE BRENT,$/BBL,67.62,68.11,67.26
1854,2018-01-04,ICE BRENT,$/BBL,68.07,68.27,67.62
1855,2018-01-03,ICE BRENT,$/BBL,67.84,67.94,66.40


In [22]:
CashFlowDF = pd.read_csv("DataSet/cash_flow_train.csv")
CashFlowDF["Inflows- currency"] = CashFlowDF["Inflows- currency"].replace(np.nan,0)
CashFlowDF["Date"] = pd.to_datetime(CashFlowDF["Date"])
CashFlowDF["Date"] = pd.DatetimeIndex(CashFlowDF["Date"].dt.strftime("%Y-%m-%d"))
CashFlowDF.set_index("Date", inplace=True)
CashFlowDF = CashFlowDF.reset_index()
CashFlowDF

,Date,Total Inflows,Customers - DDS,Customers - EFT,T&S Collections,FX Sales,Other operations,Total Outflows,Tüpraş,Other Oil,Gas,Import payments (FX purchases),Tax,Operatioınal and Admin. Expenses,VIS Buyback Payments,Net Cashflow from Operations,Inflows- currency
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,-1.109963e+06,-29274250.0,-1.048839e+05,-1.112469e+06,0.000000e+00,-2.884543e+07,0.0
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,-2.798575e+05,-7746.0,-7.887853e+05,-6.044857e+05,0.000000e+00,-3.823007e+06,0.0
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,0.000000e+00,-49032605.0,-2.936467e+06,-7.175890e+06,-2.612621e+07,-8.395180e+07,0.0
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,-1.406590e+06,-12049750.0,-8.218624e+05,-2.906375e+06,0.000000e+00,9.385578e+05,0.0
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,-4.078995e+05,-67702.5,-4.438174e+06,-1.030241e+06,0.000000e+00,1.808252e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,-2.055985e+06,-70612500.0,-7.034186e+06,-2.771415e+06,-1.500000e+06,9.034332e+06,12500.0
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,-1.551287e+06,-70621875.0,-9.880153e+06,-9.688081e+06,-2.030862e+06,-1.715431e+08,0.0
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,-1.982722e+07,-70618750.0,-9.362510e+06,-1.519839e+07,0.000000e+00,2.001448e+08,0.0
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,-2.591450e+06,-70881250.0,-4.370705e+06,-7.624322e+06,0.000000e+00,1.136393e+08,0.0


In [23]:
PlattsDF = pd.read_csv("DataSet/platts.csv")
PlattsDF["Tarih"] = pd.to_datetime(PlattsDF['Tarih'])
PlattsDF['Tarih'] = pd.DatetimeIndex(PlattsDF["Tarih"].dt.strftime("%Y-%m-%d"))

PlattsDF = PlattsDF.iloc[:,:-1].copy()
PlattsDF.rename(columns={"Tarih":"Date"}, inplace=True)
PlattsDF = PlattsDF[::-1]
PlattsDF.set_index("Date", inplace=True)
Platts10PPMDF = PlattsDF[PlattsDF["Ürün"] == "10 ppm ULSD CIF Med (Genova/Lavera)" ]
PlattsPremunl10PPMDF = PlattsDF[PlattsDF["Ürün"] == "Prem Unl 10 ppm CIF Med (Genova/Lavera)"]
PlattsDF

,Ürün,Avrupa Birliği Birimi,AB Piyasa Fiyatı,AB Piyasa Fiyatı- Yüksek,AB Piyasa Fiyatı- Düşük
Date,,,,,
2018-01-02,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,632.75,633.00,632.50
2018-01-03,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,637.25,637.50,637.00
2018-01-04,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,633.25,633.50,633.00
2018-01-05,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,628.50,628.75,628.25
2018-01-06,Prem Unl 10 ppm CIF Med (Genova/Lavera),$/TON,628.50,628.75,628.25
...,...,...,...,...,...
2022-12-27,10 ppm ULSD CIF Med (Genova/Lavera),$/TON,920.00,920.25,919.75
2022-12-28,10 ppm ULSD CIF Med (Genova/Lavera),$/TON,926.25,926.50,926.00
2022-12-29,10 ppm ULSD CIF Med (Genova/Lavera),$/TON,927.00,927.25,926.75


In [24]:
Platts10PPMDF = Platts10PPMDF.reset_index()
Platts10PPMDF = Platts10PPMDF[["Date","Ürün","AB Piyasa Fiyatı","AB Piyasa Fiyatı- Yüksek","AB Piyasa Fiyatı- Düşük"]]
Platts10PPMDF.columns = ["ds","unique_id","y","High","Low",]
Platts10PPMDF

,ds,unique_id,y,High,Low
0,2018-01-02,10 ppm ULSD CIF Med (Genova/Lavera),599.00,599.25,598.75
1,2018-01-03,10 ppm ULSD CIF Med (Genova/Lavera),609.00,609.25,608.75
2,2018-01-04,10 ppm ULSD CIF Med (Genova/Lavera),610.00,610.25,609.75
3,2018-01-05,10 ppm ULSD CIF Med (Genova/Lavera),606.50,606.75,606.25
4,2018-01-06,10 ppm ULSD CIF Med (Genova/Lavera),606.50,606.75,606.25
...,...,...,...,...,...
1820,2022-12-27,10 ppm ULSD CIF Med (Genova/Lavera),920.00,920.25,919.75
1821,2022-12-28,10 ppm ULSD CIF Med (Genova/Lavera),926.25,926.50,926.00
1822,2022-12-29,10 ppm ULSD CIF Med (Genova/Lavera),927.00,927.25,926.75
1823,2022-12-30,10 ppm ULSD CIF Med (Genova/Lavera),932.25,932.50,932.00


In [25]:
Platts10PPM_1_ModelDF = Platts10PPMDF[["ds","unique_id","y"]]
Platts10PPM_2_ModelDF = Platts10PPMDF[["ds","unique_id","High"]]
Platts10PPM_2_ModelDF.columns = ["ds","unique_id","y"]
Platts10PPM_3_ModelDF = Platts10PPMDF[["ds","unique_id","Low"]]
Platts10PPM_3_ModelDF.columns = ["ds","unique_id","y"]

#### Paltts Future Estimation

In [26]:
PlattsModels = [XGBRegressor(random_state=0, n_estimators=100)]

PlattsEstimationModel = MLForecast(models=PlattsModels,
                   freq='D',
                   lags=[1,7,14],
                   lag_transforms={
                       1: [(rolling_mean, 7), (rolling_max, 7), (rolling_min, 7)],
                   },
                   date_features=['dayofweek', 'month'],
                   num_threads=6)


PlattsEstimationModel.fit(Platts10PPM_1_ModelDF, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

MLForecast(models=[XGBRegressor], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'rolling_mean_lag1_window_size7', 'rolling_max_lag1_window_size7', 'rolling_min_lag1_window_size7'], date_features=['dayofweek', 'month'], num_threads=6)

In [27]:
Platts10PPM_1_prediciton =  PlattsEstimationModel.predict(horizon=32)
Platts10PPM_1_prediciton.columns = ["unique_id","ds","y"]
Platts10PPM_1_prediciton

,unique_id,ds,y
0,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-01,968.273926
1,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-02,1043.256348
2,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-03,1118.492676
3,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-04,1172.205078
4,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-05,1211.795044
5,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-06,1262.173706
6,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-07,1259.901489
7,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-08,1264.131958
8,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-09,1226.939819
9,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-10,1165.881958


In [28]:
PlattsEstiationModel2 = MLForecast(models=PlattsModels,
                   freq='D',
                   lags=[1,7,14],
                   lag_transforms={
                       1: [(rolling_mean, 7), (rolling_max, 7), (rolling_min, 7)],
                   },
                   date_features=['dayofweek', 'month'],
                   num_threads=6)


PlattsEstiationModel2.fit(Platts10PPM_2_ModelDF, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

MLForecast(models=[XGBRegressor], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'rolling_mean_lag1_window_size7', 'rolling_max_lag1_window_size7', 'rolling_min_lag1_window_size7'], date_features=['dayofweek', 'month'], num_threads=6)

In [29]:
Platts10PPM2Prediciton =  PlattsEstiationModel2.predict(horizon=32)
Platts10PPM2Prediciton.columns = ["unique_id","ds","y"]
Platts10PPM2Prediciton

,unique_id,ds,y
0,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-01,968.523621
1,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-02,1043.506226
2,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-03,1118.743164
3,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-04,1172.455444
4,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-05,1212.045776
5,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-06,1262.424438
6,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-07,1260.152222
7,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-08,1264.382812
8,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-09,1227.192261
9,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-10,1166.130859


In [30]:
PlattsEstimationModel3 = MLForecast(models=PlattsModels,
                   freq='D',
                   lags=[1,7,14],
                   lag_transforms={
                       1: [(rolling_mean, 7), (rolling_max, 7), (rolling_min, 7)],
                   },
                   date_features=['dayofweek', 'day'],
                   num_threads=6)


PlattsEstimationModel3.fit(Platts10PPM_3_ModelDF, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

MLForecast(models=[XGBRegressor], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'rolling_mean_lag1_window_size7', 'rolling_max_lag1_window_size7', 'rolling_min_lag1_window_size7'], date_features=['dayofweek', 'day'], num_threads=6)

In [31]:
Platts10PPM3Prediciton =  PlattsEstimationModel3.predict(horizon=32)
Platts10PPM3Prediciton.columns = ["unique_id","ds","y"]
Platts10PPM3Prediciton

,unique_id,ds,y
0,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-01,941.376587
1,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-02,925.596313
2,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-03,918.014648
3,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-04,918.810486
4,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-05,916.785706
5,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-06,899.827209
6,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-07,893.262146
7,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-08,893.290100
8,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-09,898.858948
9,10 ppm ULSD CIF Med (Genova/Lavera),2023-01-10,894.941345


In [32]:
Platts10PPM_1_ModelDF = pd.concat([Platts10PPM_1_ModelDF,Platts10PPM_1_prediciton],ignore_index = True)
Platts10PPM_1_ModelDF.columns = ["ds","unique_id","AB Piyasa Fiyatı"]
Platts10PPM_2_ModelDF = pd.concat([Platts10PPM_2_ModelDF,Platts10PPM2Prediciton],ignore_index = True)
Platts10PPM_2_ModelDF.columns = ["ds","unique_id","AB Piyasa Fiyatı-Yüksek"]
Platts10PPM_3_ModelDF = pd.concat([Platts10PPM_3_ModelDF,Platts10PPM3Prediciton],ignore_index = True)
Platts10PPM_3_ModelDF.columns = ["ds","unique_id","AB Piyasa Fiyatı-Düşük"]

In [33]:
Platts10PPMDF = pd.merge(Platts10PPM_1_ModelDF, Platts10PPM_2_ModelDF,on=["ds","unique_id"], how='inner')
Platts10PPMDF = pd.merge(Platts10PPMDF,Platts10PPM_3_ModelDF,on=["ds","unique_id"], how='inner')
Platts10PPMDF = Platts10PPMDF[["ds","AB Piyasa Fiyatı","AB Piyasa Fiyatı-Yüksek","AB Piyasa Fiyatı-Düşük"]]
Platts10PPMDF.columns = ["Date","AB Piyasa FiyatıPP","AB Piyasa Fiyatı-YüksekPP","AB Piyasa Fiyatı-DüşükPP"]
Platts10PPMDF

,Date,AB Piyasa FiyatıPP,AB Piyasa Fiyatı-YüksekPP,AB Piyasa Fiyatı-DüşükPP
0,2018-01-02,599.000000,599.250000,598.750000
1,2018-01-03,609.000000,609.250000,608.750000
2,2018-01-04,610.000000,610.250000,609.750000
3,2018-01-05,606.500000,606.750000,606.250000
4,2018-01-06,606.500000,606.750000,606.250000
...,...,...,...,...
1852,2023-01-28,1128.479736,1128.730103,925.373535
1853,2023-01-29,1146.456055,1146.706055,926.428162
1854,2023-01-30,1169.015137,1169.265259,924.799133
1855,2023-01-31,1132.180908,1132.431519,925.010742


In [34]:
PlattsPremunl10PPMDF = PlattsPremunl10PPMDF.reset_index()
PlattsPremunl10PPMDF = PlattsPremunl10PPMDF[["Date","Ürün","AB Piyasa Fiyatı","AB Piyasa Fiyatı- Yüksek","AB Piyasa Fiyatı- Düşük"]]
PlattsPremunl10PPMDF.columns = ["ds","unique_id","y","High","Low",]

PlattsPremunl10PPM_1_ModelDF = PlattsPremunl10PPMDF[["ds","unique_id","y"]]
PlattsPremunl10PPM_2_ModelDF = PlattsPremunl10PPMDF[["ds","unique_id","High"]]
PlattsPremunl10PPM_2_ModelDF.columns = ["ds","unique_id","y"]
PlattsPremunl10PPM_3_ModelDF = PlattsPremunl10PPMDF[["ds","unique_id","Low"]]
PlattsPremunl10PPM_3_ModelDF.columns = ["ds","unique_id","y"]

PlattsModelList1 = [PlattsPremunl10PPM_1_ModelDF,PlattsPremunl10PPM_2_ModelDF,PlattsPremunl10PPM_3_ModelDF]

PlattsEstimationModel = MLForecast(models=PlattsModels,
                    freq='D',
                    lags=[1,7,14],
                    lag_transforms={
                        1: [(rolling_mean, 7), (rolling_max, 7), (rolling_min, 7)],
                    },
                    date_features=['dayofweek', 'day'],
                    num_threads=6)


Plattes_Models_1 = [] 
for i in PlattsModelList1:
    models_list = PlattsEstimationModel.fit(i, id_col='unique_id', time_col='ds', target_col='y', static_features=[])
    Plattes_Models_1.append(models_list)

PlattsPremunl10PPM_1_prediciton =  Plattes_Models_1[0].predict(horizon=32)
PlattsPremunl10PPM_1_prediciton.columns = ["unique_id","ds","y"]
PlattsPremunl10PPM_2_prediciton =  Plattes_Models_1[1].predict(horizon=32)
PlattsPremunl10PPM_2_prediciton.columns = ["unique_id","ds","y"]
PlattsPremunl10PPM_3_prediciton =  Plattes_Models_1[2].predict(horizon=32)
PlattsPremunl10PPM_3_prediciton.columns = ["unique_id","ds","y"]

PlattsPremunl10PPM_1_ModelDF = pd.concat([PlattsPremunl10PPM_1_ModelDF,PlattsPremunl10PPM_1_prediciton],ignore_index = True)
PlattsPremunl10PPM_1_ModelDF.columns = ["ds","unique_id","AB Piyasa Fiyatı"]
PlattsPremunl10PPM_2_ModelDF = pd.concat([PlattsPremunl10PPM_2_ModelDF,PlattsPremunl10PPM_2_prediciton],ignore_index = True)
PlattsPremunl10PPM_2_ModelDF.columns = ["ds","unique_id","AB Piyasa Fiyatı-Yüksek"]
PlattsPremunl10PPM_3_ModelDF = pd.concat([PlattsPremunl10PPM_3_ModelDF,PlattsPremunl10PPM_3_prediciton],ignore_index = True)
PlattsPremunl10PPM_3_ModelDF.columns = ["ds","unique_id","AB Piyasa Fiyatı-Düşük"]

PlattsPremunl10PPMDF = pd.merge(PlattsPremunl10PPM_1_ModelDF, PlattsPremunl10PPM_2_ModelDF,on=["ds","unique_id"], how='inner')
PlattsPremunl10PPMDF = pd.merge(PlattsPremunl10PPMDF,PlattsPremunl10PPM_3_ModelDF,on=["ds","unique_id"], how='inner')
PlattsPremunl10PPMDF= PlattsPremunl10PPMDF[["ds","AB Piyasa Fiyatı","AB Piyasa Fiyatı-Yüksek","AB Piyasa Fiyatı-Düşük"]]
PlattsPremunl10PPMDF.columns = ["Date","AB Piyasa FiyatıPPrem","AB Piyasa Fiyatı-YüksekPPrem","AB Piyasa Fiyatı-DüşükPPrem"]


PlattsDF = PlattsPremunl10PPMDF.merge(Platts10PPMDF, on=["Date"], how="inner")
PlattsDF

,Date,AB Piyasa FiyatıPPrem,AB Piyasa Fiyatı-YüksekPPrem,AB Piyasa Fiyatı-DüşükPPrem,AB Piyasa FiyatıPP,AB Piyasa Fiyatı-YüksekPP,AB Piyasa Fiyatı-DüşükPP
0,2018-01-02,632.750000,633.000000,632.500000,599.000000,599.250000,598.750000
1,2018-01-03,637.250000,637.500000,637.000000,609.000000,609.250000,608.750000
2,2018-01-04,633.250000,633.500000,633.000000,610.000000,610.250000,609.750000
3,2018-01-05,628.500000,628.750000,628.250000,606.500000,606.750000,606.250000
4,2018-01-06,628.500000,628.750000,628.250000,606.500000,606.750000,606.250000
...,...,...,...,...,...,...,...
1852,2023-01-28,875.235962,875.235962,875.235962,1128.479736,1128.730103,925.373535
1853,2023-01-29,874.004883,874.004883,874.004883,1146.456055,1146.706055,926.428162
1854,2023-01-30,865.536621,865.536621,865.536621,1169.015137,1169.265259,924.799133
1855,2023-01-31,867.536377,867.536377,867.536377,1132.180908,1132.431519,925.010742


In [35]:
CurrencyDF = pd.read_csv("DataSet/usd.csv")
CurrencyDF["Tarih"] = pd.to_datetime(CurrencyDF["Tarih"])
CurrencyDF["Tarih"] = pd.DatetimeIndex(CurrencyDF["Tarih"].dt.strftime("%Y-%m-%d"))
CurrencyDF.rename(columns={"Tarih":"Date"},inplace=True)
CurrencyDF.set_index("Date", inplace=True)
CurrencyDF = CurrencyDF.fillna(method="ffill")
CurrencyDF = CurrencyDF.reset_index()
CurrencyDF

,Date,Yıl,USD ALIŞ,USD SATIŞ,EUR ALIŞ,EUR SATIŞ,GBP ALIŞ,GBP SATIŞ
0,2018-01-02,2018,3.7719,3.7787,4.5155,4.5237,5.0803,5.1068
1,2018-01-03,2018,3.7652,3.7719,4.5375,4.5457,5.0910,5.1175
2,2018-01-04,2018,3.7617,3.7685,4.5267,4.5349,5.1014,5.1280
3,2018-01-05,2018,3.7600,3.7668,4.5269,4.5351,5.0818,5.1083
4,2018-01-06,2018,3.7600,3.7668,4.5269,4.5351,5.0818,5.1083
...,...,...,...,...,...,...,...,...
1852,2023-01-28,2023,18.7853,18.8191,20.4931,20.5301,23.2460,23.3671
1853,2023-01-29,2023,18.7853,18.8191,20.4931,20.5301,23.2460,23.3671
1854,2023-01-30,2023,18.7852,18.8191,20.4401,20.4769,23.2050,23.3260
1855,2023-01-31,2023,18.7876,18.8215,20.4525,20.4894,23.2356,23.3568


# Data Merge

In [36]:
AllDF = pd.merge(CashFlowDF, CurrencyDF,on="Date", how='inner')
AllDF = pd.merge(AllDF,BrentDF,on="Date",how='inner')
AllDF 

,Date,Total Inflows,Customers - DDS,Customers - EFT,T&S Collections,FX Sales,Other operations,Total Outflows,Tüpraş,Other Oil,...,USD SATIŞ,EUR ALIŞ,EUR SATIŞ,GBP ALIŞ,GBP SATIŞ,Ürün,Avrupa Birliği Para Birimi,AB Piyasa Fiyatı,AB Piyasa Fiyatı- Yüksek,AB Piyasa Fiyatı- Düşük
0,2019-01-02,1.747050e+07,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,5.2905,6.0422,6.0530,6.7135,6.7485,ICE BRENT,$/BBL,54.91,56.56,52.51
1,2019-01-03,1.153048e+07,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,5.3412,6.1061,6.1171,6.7643,6.7996,ICE BRENT,$/BBL,55.95,56.30,53.93
2,2019-01-04,2.591328e+07,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,5.4686,6.2057,6.2169,6.8468,6.8825,ICE BRENT,$/BBL,57.06,58.31,55.34
3,2019-01-07,5.259345e+07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-5.165489e+07,-3.110594e+07,-3.364373e+06,...,5.4186,6.1687,6.1798,6.8370,6.8726,ICE BRENT,$/BBL,57.33,58.93,57.26
4,2019-01-08,3.650429e+07,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.842178e+07,-1.246099e+07,-1.676525e+04,...,5.3693,6.1285,6.1395,6.8184,6.8539,ICE BRENT,$/BBL,58.72,58.87,57.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,1.408741e+08,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,18.8188,20.4370,20.4738,23.0914,23.2118,ICE BRENT,$/BBL,87.47,88.01,85.74
1021,2023-01-27,1.062562e+08,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,18.8191,20.4931,20.5301,23.2460,23.3671,ICE BRENT,$/BBL,86.66,88.95,85.69
1022,2023-01-30,5.225381e+08,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,18.8191,20.4401,20.4769,23.2050,23.3260,ICE BRENT,$/BBL,84.90,87.48,84.66
1023,2023-01-31,2.323393e+08,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,18.8215,20.4525,20.4894,23.2356,23.3568,ICE BRENT,$/BBL,84.49,85.25,83.73


In [37]:
AllDF['Date'] = pd.to_datetime(AllDF['Date'])
PlattsDF['Date'] = pd.to_datetime(PlattsDF['Date'])
MergedDF = AllDF.merge(PlattsDF, on='Date', how='inner')

## Target Data Creation

"Total Inflows","Total Outflows"

In [19]:
TargetModelDF = MergedDF[["Date",'Customers - DDS','Customers - EFT','T&S Collections','FX Sales','Other operations','Tüpraş',
                'Other Oil','Gas','Import payments (FX purchases)','Tax','Operatioınal and Admin. Expenses','VIS Buyback Payments','Net Cashflow from Operations',
                'Inflows- currency','USD ALIŞ','USD SATIŞ','EUR ALIŞ','EUR SATIŞ', 'GBP ALIŞ', 'GBP SATIŞ','Ürün',
                'AB Piyasa Fiyatı','AB Piyasa Fiyatı- Yüksek', 'AB Piyasa Fiyatı- Düşük','AB Piyasa FiyatıPP', 'AB Piyasa Fiyatı-YüksekPP','AB Piyasa Fiyatı-DüşükPP',
                "AB Piyasa FiyatıPPrem","AB Piyasa Fiyatı-YüksekPPrem","AB Piyasa Fiyatı-DüşükPPrem"]].copy()
TargetModelDF.rename(columns={"Date":"ds","Ürün":"unique_id","Net Cashflow from Operations":"y"}, inplace= True)
TargetModelDF["unique_id"] = "Product"
TargetModelDF

,ds,Customers - DDS,Customers - EFT,T&S Collections,FX Sales,Other operations,Tüpraş,Other Oil,Gas,Import payments (FX purchases),...,unique_id,AB Piyasa Fiyatı,AB Piyasa Fiyatı- Yüksek,AB Piyasa Fiyatı- Düşük,AB Piyasa FiyatıPP,AB Piyasa Fiyatı-YüksekPP,AB Piyasa Fiyatı-DüşükPP,AB Piyasa FiyatıPPrem,AB Piyasa Fiyatı-YüksekPPrem,AB Piyasa Fiyatı-DüşükPPrem
0,2019-01-02,7718750.0,9.655500e+06,0.000000e+00,0.0,96250.0,-1.457022e+07,-1.441510e+05,-1.109963e+06,-29274250.0,...,Shell,54.91,56.56,52.51,521.250000,521.500000,521.000000,485.500000,485.750000,485.250000
1,2019-01-03,3431500.0,8.098978e+06,0.000000e+00,0.0,0.0,-1.337181e+07,-3.007992e+05,-2.798575e+05,-7746.0,...,Shell,55.95,56.30,53.93,517.500000,517.750000,517.250000,475.250000,475.500000,475.000000
2,2019-01-04,13775750.0,1.213753e+07,0.000000e+00,0.0,0.0,-2.459390e+07,0.000000e+00,0.000000e+00,-49032605.0,...,Shell,57.06,58.31,55.34,534.500000,534.750000,534.250000,488.000000,488.250000,487.750000
3,2019-01-07,39041750.0,1.355170e+07,0.000000e+00,0.0,0.0,-3.110594e+07,-3.364373e+06,-1.406590e+06,-12049750.0,...,Shell,57.33,58.93,57.26,549.750000,550.000000,549.500000,496.500000,496.750000,496.250000
4,2019-01-08,15137500.0,2.136679e+07,0.000000e+00,0.0,0.0,-1.246099e+07,-1.676525e+04,-4.078995e+05,-67702.5,...,Shell,58.72,58.87,57.13,551.250000,551.500000,551.000000,491.750000,492.000000,491.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-01-26,90239000.0,4.977288e+07,8.622500e+05,0.0,0.0,-4.748971e+07,-3.760017e+05,-2.055985e+06,-70612500.0,...,Shell,87.47,88.01,85.74,1141.204712,1141.455200,1140.954102,890.988831,890.988831,890.988831
1021,2023-01-27,53296500.0,5.282375e+07,1.359018e+05,0.0,0.0,-4.800505e+07,-1.360219e+08,-1.551287e+06,-70621875.0,...,Shell,86.66,88.95,85.69,1142.208496,1142.458618,1141.958374,874.723022,874.723022,874.723022
1022,2023-01-30,350467750.0,1.679393e+08,4.131052e+06,0.0,0.0,-1.854239e+08,-2.196258e+07,-1.982722e+07,-70618750.0,...,Shell,84.90,87.48,84.66,1169.015137,1169.265259,1168.765381,863.304565,863.304565,863.304565
1023,2023-01-31,131395500.0,9.922482e+07,1.718965e+06,0.0,0.0,-3.198080e+07,-1.251479e+06,-2.591450e+06,-70881250.0,...,Shell,84.49,85.25,83.73,1132.180908,1132.431519,1131.930542,865.528931,865.528931,865.528931


# LSTM Model Improvement

In [20]:
Estimation_size = TargetModelDF.index.nunique()
Models = [LSTM(h=Estimation_size,input_size=-1,
               loss=DistributionLoss(distribution='Normal', level=[90]),
               max_steps=150,
               encoder_n_layers=2,
               encoder_hidden_size=200,
               context_size=10,
               decoder_hidden_size=200,
               decoder_layers=2,
               learning_rate=1e-4,
               scaler_type='standard',
               futr_exog_list=['y_[lag12]'])]

TargetModel = NeuralForecast(models=Models, freq='D')

Global seed set to 1


Please Don't use train split for getting the better result !!!!

In [21]:
TrainModel = TargetModelDF[:-23]
TestModel = TargetModelDF[-23:]

In [22]:
TargetModel.fit(TrainModel)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## Prediction

In [2]:
SaveDF = pd.read_csv("DataSet/Save.csv")
SaveDF

,Date,Net Cashflow from Operations
0,2023-02-02,0
1,2023-02-03,0
2,2023-02-06,0
3,2023-02-07,0
4,2023-02-08,0
...,...,...
65,2023-05-08,0
66,2023-05-09,0
67,2023-05-10,0
68,2023-05-11,0


In [24]:
ModelPrediction = TargetModel.predict(TestModel).reset_index()
ModelPrediction

Predicting: 0it [00:00, ?it/s]

,unique_id,ds,LSTM,LSTM-median,LSTM-lo-90,LSTM-hi-90
0,Shell,2023-02-02,-70122080.0,-70355760.0,-137960400.0,-5804486.5
1,Shell,2023-02-03,-53580336.0,-53951472.0,-122338848.0,14168165.0
2,Shell,2023-02-04,-23754482.0,-24562208.0,-73873272.0,27477930.0
3,Shell,2023-02-05,-4858488.0,-4591935.0,-50678108.0,40337832.0
4,Shell,2023-02-06,-12583959.0,-11652930.0,-62724040.0,34646952.0
...,...,...,...,...,...,...
1020,Shell,2025-11-18,8265154.5,11233870.0,-183023280.0,196428688.0
1021,Shell,2025-11-19,15709705.0,13744765.0,-140273952.0,182774736.0
1022,Shell,2025-11-20,17128818.0,17309216.0,-174823120.0,205078720.0
1023,Shell,2025-11-21,42562788.0,44464104.0,-183699808.0,264495680.0


### MAE 

In [25]:
def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))

In [26]:
LSTMResult = mae(TargetModelDF["y"],ModelPrediction["LSTM"])
LSTMResult

41842143.59317276

In [27]:
LSTMMedianResult = mae(TargetModelDF["y"],ModelPrediction["LSTM-median"])
LSTMMedianResult

41697534.86230691

# Last Target Data Chosen

In [28]:
PredicitionLimittation = ModelPrediction[:100]

In [29]:
ModelPredicitionNew = PredicitionLimittation
ModelPredicitionNew.rename(columns={"ds":"Date"}, inplace=True)
ModelPredicitionNew["Date"] = ModelPredicitionNew["Date"].dt.strftime("%Y-%m-%d")

SaveDf_new = ModelPredicitionNew[["Date","LSTM-median"]]
SaveDf_new

,Date,LSTM-median
0,2023-02-02,-70355760.0
1,2023-02-03,-53951472.0
2,2023-02-04,-24562208.0
3,2023-02-05,-4591935.0
4,2023-02-06,-11652930.0
...,...,...
65,2023-04-08,9930927.0
66,2023-04-09,27095200.0
67,2023-04-10,16515226.0
68,2023-04-11,29159368.0


In [30]:
TargetModel_SaveDF = pd.merge(SaveDf_new,SaveDF, left_index=True,right_index=True, how="inner")
TargetModel_SaveDF = TargetModel_SaveDF[["Date_y","LSTM-median"]]
TargetModel_SaveDF.columns = ["Date","Net Cashflow from Operations"]
TargetModel_SaveDF

,Date,Net Cashflow from Operations
0,2023-02-02,-70355760.0
1,2023-02-03,-53951472.0
2,2023-02-06,-24562208.0
3,2023-02-07,-4591935.0
4,2023-02-08,-11652930.0
...,...,...
65,2023-05-08,9930927.0
66,2023-05-09,27095200.0
67,2023-05-10,16515226.0
68,2023-05-11,29159368.0
